In [10]:
import pyopencl as cl
from pyopencl.tools import get_test_platforms_and_devices
import numpy as np

import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from datetime import datetime

In [11]:
get_test_platforms_and_devices()

[(<pyopencl.Platform 'Apple' at 0x7fff0000>,
  [<pyopencl.Device 'Intel(R) Core(TM) i7-7920HQ CPU @ 3.10GHz' on 'Apple' at 0xffffffff>,
   <pyopencl.Device 'Intel(R) HD Graphics 630' on 'Apple' at 0x1024500>,
   <pyopencl.Device 'AMD Radeon Pro 560 Compute Engine' on 'Apple' at 0x1021c00>])]

In [12]:
# os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'
# os.environ['PYOPENCL_CTX'] = '1'
# 
# (n, m, p) = (3, 4, 5)
# 
# # a = np.random.randn(n, m).astype(np.float32)
# # b = np.random.randn(m, p).astype(np.float32)
# a = np.random.randint(2, size=(n*m))
# b = np.random.randint(2, size=(m*p))
# c = np.zeros((n*p), dtype=np.float32)
# 
# a = a.astype(np.float32)
# b = b.astype(np.float32)


In [6]:

os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'
os.environ['PYOPENCL_CTX'] = '1'

(n, m, p) = (2048, 2048, 2048)

# a = np.random.randn(n, m).astype(np.float32)
# b = np.random.randn(m, p).astype(np.float32)
a = np.random.randint(2, size=(n*m))
b = np.random.randint(2, size=(m*p))
c = np.zeros((n*p), dtype=np.float32)

a = a.astype(np.float32)
b = b.astype(np.float32)

In [18]:

platform = cl.get_platforms()
my_gpu_devices = platform[0].get_devices(device_type=cl.device_type.GPU)
my_gpu_devices
# ctx = cl.Context(devices=my_gpu_devices)


TypeError: get_info(): incompatible function arguments. The following argument types are supported:
    1. (self: pyopencl._cl.Device, arg0: int) -> object

Invoked with: 'IMAGE_SUPPORT'

TypeError: get_info(): incompatible function arguments. The following argument types are supported:
    1. (self: pyopencl._cl.Device, arg0: int) -> object

Invoked with: 'IMAGE_SUPPORT'

In [26]:
#setup of context queue and buffers
#ctx = cl.create_some_context()

my_gpu_devices = platform[0].get_devices(device_type=cl.device_type.GPU)
ctx = cl.Context(devices=my_gpu_devices)

queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_buf = cl.Buffer\
   (ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
b_buf = cl.Buffer\
   (ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
c_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)

In [ ]:
prg = cl.Program(ctx, """
    __kernel void multiply(ushort n,
                            ushort m, 
                            ushort p, 
                            __global float *a,
                            __global float *b, 
                            __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = 0.0f;
      int rowC = gid/p;
      int colC = gid%p;
      __global float *pA = &a[rowC*m];
      __global float *pB = &b[colC];
      for(int k=0; k<m; k++)
      {
         pB = &b[colC+k*p];
         c[gid] += (*(pA++))*(*pB);
      }
    }
    """).build()


In [ ]:
prg_sqrt = cl.Program(ctx, """
    __kernel void squareroot(ushort n, 
    ushortm, ushortp, 
""")


In [10]:
# prg.multiply(queue, c.shape, None,
#              np.uint16(n), np.uint16(m), np.uint16(p),
#              a_buf, b_buf, c_buf)
# 
# a_mul_b = np.empty_like(c)
# cl.enqueue_copy(queue, a_mul_b, c_buf)
# 
# print("matrix A:")
# print(a.reshape(n, m))
# print("matrix B:")
# print(b.reshape(m, p))
# print("multiplied A*B:")
# print(a_mul_b.reshape(n, p))

matrix A:
[[1. 1. 0. ... 0. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 1. 1. 0.]]
matrix B:
[[0. 0. 1. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 0. 0. 1.]
 ...
 [0. 0. 1. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]]
multiplied A*B:
[[483. 534. 493. ... 478. 495. 516.]
 [512. 551. 512. ... 498. 516. 540.]
 [474. 547. 506. ... 493. 515. 530.]
 ...
 [517. 535. 508. ... 491. 520. 525.]
 [497. 528. 513. ... 495. 511. 520.]
 [522. 558. 517. ... 517. 526. 551.]]


In [28]:
#%timeit -n 100 prg.multiply(queue, c.shape, None, np.uint16(n), np.uint16(m), np.uint16(p), a_buf, b_buf, c_buf)
start = datetime.now()
prg.multiply(queue, c.shape, None, np.uint16(n), np.uint16(m), np.uint16(p), a_buf, b_buf, c_buf)
stop = datetime.now()
print("time is = "+str((stop-start).microseconds))

a_mul_b = np.empty_like(c)
cl.enqueue_copy(queue, a_mul_b, c_buf)

print("matrix A:")
print(a.reshape(n, m))
print("matrix B:")
print(b.reshape(m, p))
print("multiplied A*B:")
print(a_mul_b.reshape(n, p))

time is = 2799


matrix A:
[[1. 1. 0. ... 1. 1. 1.]
 [1. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 1. 0. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 0. 1.]
 [0. 0. 1. ... 1. 1. 0.]]
matrix B:
[[1. 1. 0. ... 1. 1. 0.]
 [1. 1. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [1. 1. 0. ... 0. 0. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [0. 1. 0. ... 1. 1. 0.]]
multiplied A*B:
[[518. 528. 505. ... 497. 510. 518.]
 [503. 499. 490. ... 487. 490. 499.]
 [528. 516. 489. ... 504. 494. 539.]
 ...
 [533. 523. 519. ... 487. 516. 524.]
 [543. 527. 522. ... 500. 547. 540.]
 [504. 513. 507. ... 510. 494. 506.]]


In [11]:
start = datetime.now()
prg.multiply(queue, c.shape, None, np.uint16(n), np.uint16(m), np.uint16(p), a_buf, b_buf, c_buf)
stop = datetime.now()
print("time is = "+str((stop-start).microseconds))


(4194304,)

(2048, 2048)


In [29]:
(n, m, p) = (2048, 2048, 2048)

# a = np.random.randn(n, m).astype(np.float32)
# b = np.random.randn(m, p).astype(np.float32)
d = np.random.randint(2, size=(n*m))
e = np.random.randint(2, size=(m*p))
f = np.zeros((n*p), dtype=np.float32)

d = d.astype(np.float32)
e = e.astype(np.float32)


1048126.0

time is = 3363


In [30]:
start = datetime.now()
np.dot(d,e)
stop = datetime.now()
print("time is = "+str((stop-start).microseconds))


1048126.0

time is = 3363
